In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0

Download CAEMP25N total employment (# of jobs) for all counties in TN and TN state total

In [4]:
data = pd.read_csv('../Data Downloads/BEA_2017_2022Employment.csv', dtype = str)

In [5]:
data.head()

,GeoFips,GeoName,LineCode,Category,Sub Category,Description,2017,2018,2019,2020,2021,2022
0,47000,Tennessee,NaN,All,All,Employment by place of work,NaN,NaN,NaN,NaN,NaN,NaN
1,47000,Tennessee,10,Total,Total,Total employment (number of jobs),4033011,4120203,4169016,4121045,4301472,4506394
2,47000,Tennessee,NaN,By Type,By Type,By type,NaN,NaN,NaN,NaN,NaN,NaN
3,47000,Tennessee,20,By Type,"By Type, Wage and salary",Wage and salary employment,3083963,3133993,3195762,3074162,3170189,3305841
4,47000,Tennessee,40,By Type,"By Type, Proprietors",Proprietors employment,949048,986210,973254,1046883,1131283,1200553


In [ ]:
data = data.loc[(data['Description'] != 'Employment by place of work')&(data['Description'] != 'By type')&(data['Description'] != 'By industry')]

In [ ]:
data.head()

In [ ]:
data['Category'] = data['Category'].str.strip()
data['Sub Category'] = data['Sub Category'].str.strip()
data['Description'] = data['Description'].str.strip()

In [ ]:
data = data.replace({'(D)': 666666666})
cols = ['2017', '2018', '2019', '2020', '2021', '2022']
data[cols] = data[cols].astype(float)
data = data.rename(columns = {'GeoName': 'NAME'})
data['ID'] = data['Category'] + "|" + data['Sub Category'] + "|" + data['Description']
data = data.drop(columns = ['GeoFips', 'LineCode', 'Category', 'Sub Category', 'Description'])

In [ ]:
ids = list(data['ID'].unique())
geos = list(data['NAME'].unique())
data = data.set_index(['NAME', 'ID'])

In [ ]:
data = data.transpose()

In [ ]:
#aggregate by double column-headers
for thing in ids:
    #MPO
    thelist = [data[('Davidson, TN', '{}'.format(thing))],data[('Maury, TN', '{}'.format(thing))],
               data[('Robertson, TN', '{}'.format(thing))],data[('Rutherford, TN', '{}'.format(thing))], 
               data[('Sumner, TN', '{}'.format(thing))], data[('Williamson, TN', '{}'.format(thing))], 
               data[('Wilson, TN', '{}'.format(thing))]]
    data[('MPO', '{}'.format(thing))] = sum(thelist)
    #GNRC
    thelist = [data[('Cheatham, TN', '{}'.format(thing))], data[('Davidson, TN', '{}'.format(thing))], 
               data[('Dickson, TN', '{}'.format(thing))], data[('Houston, TN', '{}'.format(thing))],
               data[('Humphreys, TN', '{}'.format(thing))], data[('Montgomery, TN', '{}'.format(thing))],
               data[('Robertson, TN', '{}'.format(thing))],data[('Rutherford, TN', '{}'.format(thing))], 
               data[('Sumner, TN', '{}'.format(thing))], data[('Stewart, TN', '{}'.format(thing))], 
               data[('Trousdale, TN', '{}'.format(thing))],
               data[('Williamson, TN', '{}'.format(thing))], data[('Wilson, TN', '{}'.format(thing))]]
    data[('GNRC', '{}'.format(thing))] = sum(thelist)
    #GNRC Region
    thelist = [data[('Cheatham, TN', '{}'.format(thing))], data[('Davidson, TN', '{}'.format(thing))], 
               data[('Dickson, TN', '{}'.format(thing))], data[('Houston, TN', '{}'.format(thing))],
               data[('Humphreys, TN', '{}'.format(thing))], data[('Montgomery, TN', '{}'.format(thing))],
               data[('Robertson, TN', '{}'.format(thing))],data[('Rutherford, TN', '{}'.format(thing))], 
               data[('Sumner, TN', '{}'.format(thing))], data[('Stewart, TN', '{}'.format(thing))], 
               data[('Trousdale, TN', '{}'.format(thing))], data[('Williamson, TN', '{}'.format(thing))], 
               data[('Wilson, TN', '{}'.format(thing))], data[('Maury, TN', '{}'.format(thing))]]
    data[('GNRC Region', '{}'.format(thing))] = sum(thelist)

In [ ]:
geos.append('GNRC')
geos.append('GNRC Region')
geos.append('MPO')

In [ ]:
#calculate percentages
for geo in geos:
    data[('{}'.format(geo), 'By Type|By Type, Wage and salary|Wage and salary employment %')] = percent(data[('{}'.format(geo), 'By Type|By Type, Wage and salary|Wage and salary employment')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By Type|By Type, Proprietors|Proprietors employment %')] = percent(data[('{}'.format(geo), 'By Type|By Type, Proprietors|Proprietors employment')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By Type|By Type, Proprietors Sub|Farm proprietors employment %')] = percent(data[('{}'.format(geo), 'By Type|By Type, Proprietors Sub|Farm proprietors employment')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By Type|By Type, Proprietors Sub|Nonfarm proprietors employment 2 %')] = percent(data[('{}'.format(geo), 'By Type|By Type, Proprietors Sub|Nonfarm proprietors employment 2')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Farm|Farm employment %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Farm|Farm employment')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm|Nonfarm employment %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm|Nonfarm employment')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Private nonfarm employment %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Private nonfarm employment')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Forestry, fishing, and related activities %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Forestry, fishing, and related activities')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Mining, quarrying, and oil and gas extraction %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Mining, quarrying, and oil and gas extraction')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Utilities %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Utilities')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Construction %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Construction')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Manufacturing %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Manufacturing')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Wholesale trade %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Wholesale trade')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Retail trade %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Retail trade')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Transportation and warehousing %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Transportation and warehousing')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Information %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Information')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Finance and insurance %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Finance and insurance')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Real estate and rental and leasing %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Real estate and rental and leasing')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Professional, scientific, and technical services %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Professional, scientific, and technical services')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Administrative and support and waste management and remediation services %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Administrative and support and waste management and remediation services')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Educational services %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Educational services')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Health care and social assistance %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Health care and social assistance')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Arts, entertainment, and recreation %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Arts, entertainment, and recreation')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Accommodation and food services %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Accommodation and food services')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Other services (except government and government enterprises) %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Nonfarm Sub|Other services (except government and government enterprises)')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Government|Government and government enterprises %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Government|Government and government enterprises')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Government Sub|Federal civilian %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Government Sub|Federal civilian')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Government Sub|Military %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Government Sub|Military')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Government Sub|State and local %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Government Sub|State and local')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Government Sub|State government %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Government Sub|State government')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])
    data[('{}'.format(geo), 'By industry|By industry, Government Sub|Local government %')] = percent(data[('{}'.format(geo), 'By industry|By industry, Government Sub|Local government')], 
                                                                                                       data[('{}'.format(geo), 'Total|Total|Total employment (number of jobs)')])

In [ ]:
data = data.rename(columns = {
    'Cheatham, TN': 'Cheatham County, Tennessee', 
    'Davidson, TN': 'Davidson County, Tennessee',
    'Dickson, TN': 'Dickson County, Tennessee',
    'Houston, TN': 'Houston County, Tennessee',
    'Humphreys, TN': 'Humphreys County, Tennessee',
    'Maury, TN': 'Maury County, Tennessee',
    'Montgomery, TN': 'Montgomery County, Tennessee',
    'Robertson, TN': 'Robertson County, Tennessee',
    'Rutherford, TN': 'Rutherford County, Tennessee',
    'Stewart, TN': 'Stewart County, Tennessee',
    'Sumner, TN': 'Sumner County, Tennessee',
    'Williamson, TN': 'Williamson County, Tennessee',
    'Wilson, TN': 'Wilson County, Tennessee',
    'Trousdale, TN': 'Trousdale County, Tennessee'
})

In [ ]:
data = data.transpose().reset_index(drop = False)
data.head()

In [ ]:
boop = data['ID'].str.split(pat = "|", expand = True)
data['Category'] = boop[0]
data['Sub Category'] = boop[1]
data['Description'] = boop[2]
data = data.drop(columns = 'ID')

In [ ]:
data['Category'] = data['Category'].str.strip()
data['Sub Category'] = data['Sub Category'].str.strip()
data['Description'] = data['Description'].str.strip()

In [ ]:
data.tail()

In [ ]:
#export to the SQLite database
conn = sq.connect('../Outputs/BEA.db')
data.to_sql('2017_2022Employment', conn, if_exists = 'replace', index = False)